# APIM ❤️ AI Agents

## Azure AI Agent Service lab

![flow](../../images/ai-agent-service.gif)

Use this playground to explore the [Azure AI Agent Service](https://learn.microsoft.com/en-us/azure/ai-services/agents/overview), leveraging Azure API Management to control multiple services, including Azure OpenAI models, Bing Web Search, Logic Apps Workflows, and OpenAPI-based APIs. This enables limitless opportunities for AI agents while maintaining control through Azure API Management!

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming convention
resource_group_location = "eastus2" # all the resources will be deployed in this location

apim_sku = 'Basicv2'

# Azure OpenAI configuration
openai_resources = [ {"name": "openai1", "location": "eastus2"} ]
openai_model_name = "gpt-4o"
openai_model_version = "2024-08-06"
openai_model_sku = "GlobalStandard"
openai_model_capacity = 200
openai_deployment_name = "gpt-4o"
openai_api_version = "2025-01-01-preview"

utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIModelSKU": { "value": openai_model_sku },
        "openAIModelCapacity": { "value": openai_model_capacity },
        "openAIAPIVersion": { "value": openai_api_version }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')    
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    project_connection_string = utils.get_deployment_output(output, 'projectConnectionString', 'AI Foundry Project Connection String', True)
    bing_search_connection = utils.get_deployment_output(output, 'bingSearchConnectionName', 'Bing Search Connection')
    weather_api_connection_id = utils.get_deployment_output(output, 'weatherAPIConnectionId', 'Weather API Connection Id')




<a id='4'></a>
### 4️⃣ List the connections

Retrieve the connections managed in AI Foundry available for the Agents

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=project_connection_string,
)

with project_client:
    connections = project_client.connections.list()
    utils.print_ok(f"Listing all connections (found {len(connections)}):")
    for connection in connections:
        utils.print_info(f"Name: {connection.name}, Type: {connection.connection_type}")


<a id='quickstart'></a>
### 🧪 Create and run a Math Tutor Agent with OpenAI Assistants API

Check the official documentation for updates on this quickstart:
https://learn.microsoft.com/en-us/azure/ai-services/agents/quickstart


In [ ]:
import time, logging
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI

with AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=project_connection_string,
    logging_enable = True
) as project_client:
    client: AzureOpenAI = project_client.inference.get_azure_openai_client(
        api_version = openai_api_version  
    )

    with client:
        agent = client.beta.assistants.create(
            model=openai_deployment_name, name="my-agent", instructions="'You are a personal math tutor. Answer questions briefly, in a sentence or less."
        )
        utils.print_ok(f"Created agent, agent ID: {agent.id}")

        thread = client.beta.threads.create()
        utils.print_ok(f"Created thread, thread ID: {thread.id}")

        message = client.beta.threads.messages.create(thread_id=thread.id, role="user", content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
            extra_headers={"x-agent-id": "agent1"}   )
        utils.print_ok(f"Created message, message ID: {message.id}")

        run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=agent.id,
            extra_headers={"x-agent-id": "agent1"}   )

        # Poll the run while run status is queued or in progress
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)  # Wait for a second
            run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
            utils.print_ok(f"Run status: {run.status}")

        client.beta.assistants.delete(agent.id)
        utils.print_ok("Deleted agent")

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print(f"🗨️ {messages.data[0].content[0].text.value}")

<a id='functions'></a>
### 🧪 Run agent operations with local function tools

👉 Check the [Azure AI Foundry Tracing](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/trace) information to understand the execution process.

In [ ]:
# The following code was copied from this sample:
# https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-projects/samples/agents/sample_agents_functions_with_azure_monitor_tracing.py

from typing import Any, Callable, Set

import time, json
from azure.ai.projects import AIProjectClient
from azure.ai.projects.telemetry import trace_function
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    utils.print_error("Application Insights was not enabled for this project.")
    utils.print_error("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    raise RuntimeError("Application Insights was not enabled for this project.")
    
configure_azure_monitor(connection_string=application_insights_connection_string)

scenario = deployment_name
tracer = trace.get_tracer(deployment_name)

# The trace_func decorator will trace the function call and enable adding additional attributes
# to the span in the function implementation. Note that this will trace the function parameters and their values.
@trace_function()
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}

    # Adding attributes to the current span
    span = trace.get_current_span()
    span.set_attribute("requested_location", location)

    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json


# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather,
}

# Initialize function tool with user function
functions = FunctionTool(functions=user_functions)

with tracer.start_as_current_span(scenario):
    with project_client:
        # Create an agent and run user's request with function calls
        agent = project_client.agents.create_agent(
            model=openai_deployment_name,
            name="my-assistant",
            instructions="You are a helpful assistant",
            tools=functions.definitions,
        )
        utils.print_ok(f"Created agent, ID: {agent.id}")

        thread = project_client.agents.create_thread()
        utils.print_ok(f"Created thread, ID: {thread.id}")

        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="Hello, what is the weather in New York and sister cities?",
        )
        utils.print_ok(f"Created message, ID: {message.id}")

        run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)
        utils.print_ok(f"Created run, ID: {run.id}")

        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)
            run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

            if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
                tool_calls = run.required_action.submit_tool_outputs.tool_calls
                if not tool_calls:
                    utils.print_error("No tool calls provided - cancelling run")
                    project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
                    break

                tool_outputs = []
                for tool_call in tool_calls:
                    if isinstance(tool_call, RequiredFunctionToolCall):
                        try:
                            output = functions.execute(tool_call)
                            tool_outputs.append(
                                ToolOutput(
                                    tool_call_id=tool_call.id,
                                    output=output,
                                )
                            )
                        except Exception as e:
                            utils.print_error(f"Error executing tool_call {tool_call.id}: {e}")

                utils.print_ok(f"Tool outputs: {tool_outputs}")
                if tool_outputs:
                    project_client.agents.submit_tool_outputs_to_run(
                        thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                    )

            utils.print_ok(f"Current run status: {run.status}")

        utils.print_ok(f"Run completed with status: {run.status}")

        # Delete the agent when done
        project_client.agents.delete_agent(agent.id)
        utils.print_ok("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print(f"🗨️ {messages.data[0].content[0].text.value}")

<a id='weatherapi'></a>
### 🧪 Run agent with Weather API from Azure API Management



In [ ]:
# The following code was copied from this sample:
# https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-projects/samples/agents/sample_agents_openapi.py

import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import OpenApiTool, OpenApiConnectionAuthDetails, OpenApiConnectionSecurityScheme
from azure.monitor.opentelemetry import configure_azure_monitor


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=project_connection_string,
)

application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    utils.print_error("Application Insights was not enabled for this project.")
    utils.print_error("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    raise RuntimeError("Application Insights was not enabled for this project.")
    
configure_azure_monitor(connection_string=application_insights_connection_string)


# [START create_agent_with_openapi]

with open("./city-weather-openapi.json", "r") as f:
    openapi_weather = jsonref.loads(f.read())

# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
auth = OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id=weather_api_connection_id))


# Initialize agent OpenApi tool using the read in OpenAPI spec
openapi_tool = OpenApiTool(
    name="get_weather", spec=openapi_weather, description="Retrieve weather information for a location", auth=auth
)


# Create agent with OpenApi tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=openai_deployment_name,
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=openapi_tool.definitions,
    )

    # [END create_agent_with_openapi]

    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What's the weather in Seattle and sister cities?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    run_steps = project_client.agents.list_run_steps(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps.data:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

                function_details = call.get("function", {})
                if function_details:
                    print(f"    Function name: {function_details.get('name')}")
        print()  # add an extra newline between steps

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

<a id='bing'></a>
### 🧪 Grounding with Bing

Check the official documentation for updates on this sample: https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding


In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=project_connection_string,
)

bing_connection = project_client.connections.get(
    connection_name=bing_search_connection
)
conn_id = bing_connection.id

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=openai_deployment_name,
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    
    utils.print_ok(f"Created agent, ID: {agent.id}")

    thread = project_client.agents.create_thread()
    utils.print_ok(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What are the top news today?",
    )
    utils.print_ok(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    utils.print_ok(f"Run finished with status: {run.status}")

    # Retrieve run step details to get Bing Search query link
    # To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
    run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
    run_steps_data = run_steps['data']

    if run.status == "failed":
        utils.print_error(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    utils.print_ok("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"🗨️ {messages.data[0].content[0].text.value}")


<a id='kql'></a>
### 🔍 Analyze Application Insights custom metrics with a KQL query

With this query you can get the custom metrics that were emitted by Azure APIM. Note that it may take a few minutes for data to become available.

In [ ]:
import pandas as pd

query = "\"" + "customMetrics \
| where name == 'Total Tokens' \
| where timestamp >= ago(1h) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) \
| extend agentID = tostring(parsedCustomDimensions.['Agent ID']) \
| summarize TotalValue = sum(value) by apimSubscription, bin(timestamp, 1m), agentID \
| order by timestamp asc" + "\""

output = utils.run(f"az monitor app-insights query --app {app_insights_name} -g {resource_group_name} --analytics-query {query}",
    f"App Insights query succeeded", f"App Insights query  failed")

table = output.json_data['tables'][0]
df = pd.DataFrame(table.get("rows"), columns = [col.get("name") for col in table.get('columns')])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%H:%M')

df


<a id='plot'></a>
### 🔍 Plot the custom metrics results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [15, 7]
if df.empty:
    print("No data to plot")
else:
    df_pivot = df.pivot(index='timestamp', columns='apimSubscription', values='TotalValue')
    ax = df_pivot.plot(kind='bar', stacked=True)
    plt.title('Total token usage over time by APIM Subscription')
    plt.xlabel('Time')
    plt.ylabel('Tokens')
    plt.legend(title='APIM Subscription')
    plt.show()

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.